In [ ]:
## Librerias
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## Path
ruta="./Datos"

In [ ]:
df=pd.DataFrame()
for i in range(len(os.listdir(ruta))):
    df_aux=pd.read_csv(os.path.join(ruta,os.listdir(ruta)[i])
                       ,sep=";",decimal=",")
    df=pd.concat([df,df_aux])
    df=df.reset_index(drop=True)

In [ ]:
for i in df.columns:
    if i !="ID":
        df[f"{i}_lag_1"]=df[i].shift(-1)
        df[f"TV_{i}_1"]=(df[i]-df[f"{i}_lag_1"])/df[f"{i}_lag_1"]
        df[f"{i}_lag_2"]=df[i].shift(-2)
        df[f"TV_{i}_2"]=(df[i]-df[f"{i}_lag_2"])/df[f"{i}_lag_2"]

In [ ]:
def Laggeo(dataframe:pd.DataFrame,column:str,n=1)->pd.DataFrame:
    """"Funcion que calcula los lag 
    de cada una de las columnas"""
    dataframe[f"{column}_lag_{str(n)}"]=df[column].shift(-n)
    return dataframe

In [ ]:
columnas=[x for x in df.columns if "ID" not in x  if not "TV_" in x]
for col in columnas:
    df[col]=df[col].fillna(df[col].mean())

In [ ]:
columnas_1=[x for x in columnas  if not "lag_" in x]

In [ ]:
for col in columnas_1:
    df[f"media_movil_{col}"]=df[col].rolling(3).mean().shift(-2)

In [ ]:
# for col in columnas_1:
#     print("="*20)
#     plt.plot(df[f"media_movil_{col}"])
#     plt.plot(df[col])
#     plt.show();

In [ ]:
for colu in df.columns:
    if colu!="ID":
        dv_Peso=df[colu].std()
        media_Peso=df[colu].mean()
        max_Peso=df[colu].max()
        min_Peso=df[colu].min()
        Coef_Variacion=dv_Peso/media_Peso
        if (Coef_Variacion>0.3) & (Coef_Variacion<0.7):
            df[colu]=(df[colu]-media_Peso)/dv_Peso
        elif (Coef_Variacion>=0.7) & (Coef_Variacion<1.3):
            df[colu]=(df[colu]-min_Peso)/(max_Peso-min_Peso)
        elif (Coef_Variacion>=1.3):
            df[colu]=np.log(df.loc[colu])

In [35]:
for i in columnas_1:
    df[f"Tendecia_{i}_categorizada"]=np.where(df[i]<df[i].shift(1),'Decreciente',np.nan)
    df[f"Tendecia_{i}_categorizada"]=np.where(df[i]==df[i].shift(1),'constante',df[f"Tendecia_{i}_categorizada"])
    df[f"Tendecia_{i}_categorizada"]=np.where(df[i]>df[i].shift(1),'Creciente',df[f"Tendecia_{i}_categorizada"])